In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl
from enceladus.workflows import TestingPipeline

# data_dir='/home/cam/Documents/database_tools/data/data-2022-11-08/mimic3/',
err_df = TestingPipeline(
    data_dir='/home/cam/Documents/database_tools/data/mimic3-data-20230205/data/',
    model_dir='heartfelt/Enceladus/model-vital-bee-206:v19',
    scaler_dir='/home/cam/Documents/database_tools/data/data-2022-11-08/mimic3/scalers_MinMax.pkl',
).run()

In [ ]:
import pickle as pkl

with open('data.pkl', 'wb') as f:
    pkl.dump(dict(abp=err_df[3], pred=err_df[4]), f)

In [ ]:
import pickle as pkl
import numpy as np
from tqdm import tqdm
from heartpy.preprocessing import flip_signal
from heartpy.peakdetection import detect_peaks
from heartpy.datautils import rolling_mean

with open('data.pkl', 'rb') as f:
    tmp = pkl.load(f)
    abp = tmp['abp'].reshape(-1, 256)
    pred = tmp['pred'].reshape(-1, 256)

fs=125
windowsize=1
ma_perc=20
pad_width = 19

series = {}
for label, windows in {'abp': abp, 'pred': pred}.items():
    sbp, dbp = [], []
    for x in tqdm(windows):
        x_pad = np.pad(x, pad_width=[pad_width, 0], constant_values=[x[0]])
        x_pad = np.pad(x_pad, pad_width=[0, pad_width], constant_values=[x[-1]])

        rol_mean = rolling_mean(x_pad, windowsize=windowsize, sample_rate=fs)
        peaks = detect_peaks(x_pad, rol_mean, ma_perc=ma_perc, sample_rate=fs)['peaklist']
        peaks = np.array(peaks) - pad_width - 1

        flip = flip_signal(x_pad)
        rol_mean = rolling_mean(flip, windowsize=windowsize, sample_rate=fs)
        valleys = detect_peaks(flip, rol_mean, ma_perc=ma_perc, sample_rate=fs)['peaklist']
        valleys = np.array(valleys) - pad_width - 1

        peak_mean = np.mean(x[peaks]) if len(peaks) > 0 else -1
        valley_mean = np.mean(x[valleys]) if len(valleys) > 0 else -1
        sbp.append(peak_mean)
        dbp.append(valley_mean)
    series[f'{label}_sbp'] = sbp
    series[f'{label}_dbp'] = dbp



In [ ]:
import pandas as pd
df = pd.DataFrame(series)
df['sbp_err'] = df['pred_sbp'] - df['abp_sbp']
df['dbp_err'] = df['pred_dbp'] - df['abp_dbp']
df['sbp_abs_err'] = np.abs(df['sbp_err'])
df['dbp_abs_err'] = np.abs(df['dbp_err'])

for err, label in zip([df['sbp_abs_err'], df['dbp_abs_err']], ['Systolic BP', 'Diastolic BP']):
    print(label)
    print(f'{len(err[err < 15]) / len(err) * 100:.{3}}% < 15mmHg')
    print(f'{len(err[err < 10]) / len(err) * 100:.{3}}% < 10mmHg')
    print(f'{len(err[err < 5]) / len(err) * 100:.{3}}% < 5mmHg')
    print()

In [ ]:
err_df = TestingPipeline(
    data_dir='/home/cam/Documents/database_tools/data/mimic3-data-20230205/data/',
    model_dir='heartfelt/Enceladus/model-vital-bee-206:v19',
    scaler_dir='/home/cam/Documents/database_tools/data/data-2022-11-08/mimic3/scalers_MinMax.pkl',
).run()

In [ ]:
import plotly.graph_objects as go

pd.options.plotting.backend = 'plotly'

fig = go.Figure()
fig.add_trace(go.Histogram(x=err_df['sbp_err'], name='SBP Error'))
fig.add_trace(go.Histogram(x=err_df['dbp_err'], name='DBP Error'))

fig.update_layout(
        barmode='overlay',
        title={
            'text': 'Model Prediction Error Histogram',
            'font': {'size': 20},
        },
        xaxis={'title': 'Error (mmHg)', 'titlefont':{'size': 15}},
        yaxis={'title': 'Number of Samples', 'titlefont':{'size': 15}},
        font={
            'family': 'Courier New, monospace',
            'color' : '#FFFFFF',
            'size'  : 12,
        },
        template='plotly_dark',
        width=800,
)
fig
fig.update_traces(opacity=0.75)

In [ ]:
# i = 18
# ppg, vpg, apg, abp, pred = None

# fig = make_subplots(rows=2, cols=1, start_cell='top-left')

# # Add inputs
# fig.add_scatter(
#     y=ppg[i, :],
#     name='PPG',
#     row=1,
#     col=1,
# )
# fig.add_scatter(
#     y=vpg[i, :] * 10,
#     name='VPG',
#     row=1,
#     col=1,
# )
# fig.add_scatter(
#     y=apg[i, :] * 50,
#     name='APG',
#     row=1,
#     col=1,
# )

# # Add ABP and prediction
# fig.add_scatter(
#     y=abp[i, :],
#     name='ABP (Ground Truth)',
#     row=2,
#     col=1
# )
# fig.add_scatter(
#     y=pred[i, :],
#     name='Prediction',
#     row=2,
#     col=1,
# )

# fig.update_layout(
#         title={
#             'text': 'Model Inputs (PPG, VPG, APG) and Module Output w/ True ABP',
#             'font': {'size': 20},
#         },
#         xaxis2={'title': 'Time (samples)', 'titlefont':{'size': 15}},
#         yaxis={'title': 'Amplitude (V)', 'titlefont':{'size': 15}},
#         yaxis2={'title': 'Amplitude (mmHg)', 'titlefont':{'size': 15}},
#         font={
#             'family': 'Courier New, monospace',
#             'color' : '#FFFFFF',
#             'size'  : 12,
#         },
#         template='plotly_dark',
#         width=800,
# )
# fig